In [28]:
import soundfile as sf
import numpy as np
import glob
import random
from tqdm import tqdm

In [9]:
male_path = '/home/jhkim21/Data/raw/sitec/SiTEC_Dict01_reading_sentence/SD01-Dict01-Ver1.2_1-2/data/male/**/*'
female_path = '/home/jhkim21/Data/raw/sitec/SiTEC_Dict01_reading_sentence/SD01-Dict01-Ver1.2_2-2/data/female/**/*'

In [10]:
male_folder = glob.glob(male_path)
female_folder = glob.glob(female_path)

In [18]:
male_folder = sorted(male_folder, key=lambda a: int(a.split('/')[-1][-3:]))
female_folder = sorted(female_folder, key=lambda a: int(a.split('/')[-1][-3:]))
print(len(male_folder), len(female_folder))

200 200


In [24]:
def concat(wav1, wav2):
    len1 = len(wav1)
    len2 = len(wav2)
    
    trim_len1 = random.randint(len1//2, len1)
    trim_len2 = random.randint(len2//2, len2)
    
    idx1 = random.randint(0, len1 - trim_len1)
    idx2 = random.randint(0, len2 - trim_len2)
    trim_wav1 = wav1[idx1:idx1+trim_len1]
    trim_wav2 = wav2[idx2:idx2+trim_len2]
    
    clean = np.concatenate((trim_wav1, trim_wav2))
    
    if random.choice([0, 1]):
        max1 = max(trim_wav1)
        gain = random.uniform(0.01, (1/max1))
        trim_wav1 *= gain
    else:
        max2 = max(trim_wav2)
        gain = random.uniform(0.01, (1/max2))
        trim_wav2 *= gain
    
    noisy = np.concatenate((trim_wav1, trim_wav2))
    
    return clean, noisy

In [19]:
train_sample = male_folder[:180] + female_folder[:180]
test_sample = male_folder[180:] + female_folder[180:]

print(len(train_sample), len(test_sample))

360 40


In [30]:
noisy_path = '/home/jhkim21/Data/processed/2022_AGC/noisy/'
clean_path = '/home/jhkim21/Data/processed/2022_AGC/clean/'

In [31]:
for i in tqdm(range(9500)):
    spk1, spk2 = random.choice(train_sample), random.choice(train_sample)
    path1, path2 = random.choice(glob.glob(spk1 + '/*.wav')), random.choice(glob.glob(spk2 + '/*.wav'))
    
    wav1, _ = sf.read(path1)
    wav2, _ = sf.read(path2)
    
    clean, noisy = concat(wav1, wav2)
    
    sf.write(noisy_path + f'gain_controlled_{i}.wav', noisy, 16000)
    sf.write(clean_path + f'original_signal_{i}.wav', clean, 16000)

100%|██████████| 9500/9500 [17:10<00:00,  9.22it/s]


In [32]:
for i in range(9500, 10000):
    spk1, spk2 = random.choice(test_sample), random.choice(test_sample)
    path1, path2 = random.choice(glob.glob(spk1 + '/*.wav')), random.choice(glob.glob(spk2 + '/*.wav'))
    
    wav1, _ = sf.read(path1)
    wav2, _ = sf.read(path2)
    
    clean, noisy = concat(wav1, wav2)
    
    sf.write(noisy_path + f'gain_controlled_{i}.wav', noisy, 16000)
    sf.write(clean_path + f'original_signal_{i}.wav', clean, 16000)